In [1]:
# General packages
import requests
import json
import pandas as pd
import numpy as np
import time
import random
import re
import string
import os
import sys
from ast import literal_eval

# Custom modules
from modules import sec_helper as sec

# Web scraping
from bs4 import BeautifulSoup as bs4


# Overview

This notebook is a great tool for extracting and exporting financial data tables and supplementary tables from SEC filings. Across mainstream financial APIs and the SEC Edgar database the options for export-ready data tables are restricted to the fundamentals: Income Statement, Cash Flow Statement, Balance Sheet.

For deeper analysis it is often of interest to look at supplementary info such as operating segment financials, leases, debt instruments, etc.

Given a company name, or list of company names, the program will retrieve the SEC index (CIK number) and pull the landing page for the requested filing.

Once the filing link has been obtained, a .get() request will pull the filing's .html and then will be converted to a BeautifulSoup object as a list of tables.

The tables are then cleaned and pre-processed as needed.

# Functions

* Functions are listed here in a single cell, also found in the modules folder of package

In [16]:
# endpoints and params 
base_sec = r'https://www.sec.gov/cgi-bin'


def get_ciks(co_list):
    """
    Get dict of CIKs with co name as key
    
    """
    # Initiatlize dictionary
    co_dict = {co: {'cik': [], 'cik_link': []} for co in co_list} 

    # loop through company names
    for co in co_dict:

        # sleep timer for each loop - randomized
        time.sleep(random.randint(2,4))


        try:

            # get_cik --> start with company name
            cik_res = get_cik(co)
            cik = cik_res[0]
            cik_link = cik_res[1]

            # append results
            co_dict[co]['cik'] = cik
            co_dict[co]['cik_link'] = cik_link


        except:

            # error here, pass
            print('error at ',co, 'pass for now')

    return co_dict


def get_cik(company_name):
    """
    Overview:
    
    Gets cik number for a single company name
    
    Params:
    
    name --> company name
    
    Returns:
    
    cik -- cik number
    link -- sec base link + cik number attached
    
    """
    # endpoint and cik params for get method
    cik_endpoint = r'https://www.sec.gov/cgi-bin/cik_lookup'
    params_cik = {'company': f'{company_name}'}

    cik_results = get_atags(get_tables(cik_endpoint, params = params_cik))

    cik = cik_results[0]
    link = base_sec + '/'+ cik_results[1] if cik_results[1] != 'none' else 'none'

    return cik, link






def get_atags(tables):
    """
    Overview: 
    gets atags for CIK lookup results
    
    params:
    bs4 tables --> results of bs4.find_all('table')
    
    """

    # only one table, grab first
    tdata = tables[0]

    # pull a tags
    atag = [data.find_all('a') for data in tdata]

    # atag[0] holds relevant tags list

    if len(atag[0])>1:
        cik = atag[0][0].text.strip()
        link = atag[0][0]['href']
        
        return cik, link
    else:
        return 'none', 'none'
    


def get_tables(endpoint, params):
    """
    Overview:
    Extracts all tables from url's html, returns a list
    
    Params:
    url
    
    """
    
    html = requests.get(endpoint, params = params).text
    
    # html = requests.get(html_url).text
    # initiate bs object
    soup = bs4(html,'lxml')
    # 25 total tables on FAA site
    tables = soup.find_all('table')
    
    return tables

def get_edgar_tables(cik):
    """
    Overview:
    Extracts all tables from sec cik page, returns a list
    
    Params:
    cik --> cik number
    
    """
    # url endpoint for browsing sec edgar
    edgar_endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"
    
    # f string cik num from parameters
    params_edgar = {'action':'getcompany',
                    'CIK':f'{cik}',
                    'type':'10-K',
                    'dateb':'',
                    'owner':'exclude',
                    'start':'',
                    'output':'',
                    'count':'100'
                   }
    
    html = requests.get(url = edgar_endpoint, params = params_edgar).text
    
    # html = requests.get(html_url).text
    # initiate bs object
    soup = bs4(html,'lxml')
    # 25 total tables on FAA site
    tables = soup.find_all('table', class_='tableFile2')
    
    return tables




def check_formd(ftype_list):
    """
    Overview:
    Takes a list of file types and checks for Form D types
    
    Params:
    List of file types
    
    Returns:
    'Yes' or 'No' string
    
    """

    if type(ftype_list) == list:
    
        d_list = [f for f in ftype_list if f == 'D' or 'D/A']

        if len(d_list) > 0:
    
            return 'Yes'
        else:
            return 'No'

    elif type(ftype_list) == str:

        if ftype_list == 'D' or 'D/A':

            return 'Yes'
        else:
            return 'No'

    else:
        return 'No'

        
def likely_private(co, filings_df):
    """
    Checks row in dataframe for a company likely to be private
    
    Returns 'yes' or 'no'
    
    """
    df = filings_df.copy()
    
    if df.at[co, 'cik'] == 'none':
        return 'Yes'
    elif '10-Q' in df.at[co, 'f_type'] or '10-K' in df.at[co, 'f_type']:
        return 'No'
    else:
        return 'Yes'
        
        
        

def get_filings(cik):
    """
    Overview:
    Takes in CIK, returns filings data
    
    Params:
    CIK --> cik number
    
    """
    
    # run function, get data table
    tables = get_edgar_tables(cik)


    # master storage for data per each filing
    file_types = []
    file_dates = []
    file_num = []
    acc_nums = []

    for row in tables[0].find_all('tr'):
        data = row.find_all('td')

        if len(data) > 1:

            # unpack data

            # append file data to master storage
            filing_type = data[0].text.strip()
            file_types.append(filing_type)
            filing_date = data[3].text.strip()
            file_dates.append(filing_date)
            filing_num = data[4].text.strip()
            file_num.append(filing_num)
            acc_num = data[2].text.strip()
            acc_s = acc_num.partition('Acc-no: ')[2]
            acc_nums.append(acc_s[:20])

    return file_types, file_dates, file_num, acc_nums
    
    
    
def get_filings_data(cik_dict):
    """
    Takes a dict of companies (keys) and CIKs (values) and returns filings response data
    
    """
    
    file_features = ['f_type', 'f_date', 'f_num', 'acc_num']

    for co in co_dict:

        # sleep timer for each loop - randomized
        time.sleep(random.randint(2,5))


        if co_dict[co]['cik'] != 'none':

            try:

                # get_filings --> uses cik number
                filing_res = get_filings(co_dict[co]['cik'])

                # unpack response variable
                f_type = filing_res[0]
                f_date = filing_res[1]
                f_num = filing_res[2]
                acc_num = filing_res[3]

                # append result variables
                co_dict[co]['f_type'] = f_type
                co_dict[co]['f_date'] = f_date
                co_dict[co]['f_num'] = f_num
                co_dict[co]['acc_num'] = acc_num

            except:

                # error here, pass for now
                print('error here at ', co, 'pass for now')

        else:

            for feat in file_features:
                co_dict[co][feat] = 'none'

    return co_dict


def filing_landing_tables(co_filing_link):
    """
    Takes a link to the SEC landing page for SPECIFIC filing
    
    """
    co_landing = requests.get(co_filing_link).text
    # Request and Get the html file 
    soup = bs4(co_landing, 'lxml')
    # retrieve all tables from BeautifulSoup object
    co_tables = soup.find_all('table')

    filing_dfs = []
    for t in co_tables:
        """
        Get the rows and table column data for all tables.
        Loop 'manually' rebuilds table data -- stripped and prettified

        """    
        if len(t) > 3:
            table_rows = t.find_all('tr')

            res = []
            for tr in table_rows:
                td = tr.find_all('td')
                row = [tr.text.strip() for tr in td if tr.text.strip()]
                if row:
                    res.append(row)
            df = pd.DataFrame(res)
            filing_dfs.append(df)
            
    return filing_dfs

def filing_homelink(co_name, file_num):
    """
    Takes a string co_name and int number of filing from filings_df and returns the html link
    
    co_name: str of company name from filings_df
    file_num: int of file number in filings_df (Chronological descending)
    
    """
    cik = filings_df['cik'][f'{co_name}']

    # Retrieve SEC Accession No.
    acc = filings_df['acc_num'][f'{co_name}'][file_num]
    acc_raw = acc.replace('-','')

    # Build data url to get html filing landing page
    sec_data_base = 'https://www.sec.gov/Archives/edgar/data/{}/{}/{}-index.htm'
    filing_homelink = sec_data_base.format(cik, acc_raw, acc)

    return filing_homelink

def get_html_path(files_table, wanted_file_type, co, file_num):
    """
    Returns the html file path from wanted file type parameter
    
    files_table: dataframe of landing page table for filing
    wanted_file_type: e.g. '10-K', '10-Q', '8-K'
    co: company name
    
    """
    # file_type
    get_file = wanted_file_type

    # Retrieve SEC Accession No.
    acc = filings_df['acc_num'][f'{co}'][file_num]
    acc_raw = acc.replace('-','')
    
    # Get the html file extension for 10-K
    extension = files_table.apply(lambda x: x[2] if x[1] == get_file else None, axis = 1)

    # filing extension
    html_path = extension[0]
    
    # build url
    sec_file_base = 'https://www.sec.gov/Archives/edgar/data/{}/{}/{}'
    cik_strip = cik.lstrip("0")

    # get url object
    sec_file_resp = requests.get(sec_file_base.format(cik_strip, acc_raw, html_path))
    

    return sec_file_resp.url


def get_filing_tables(soup_tables):
    """
    Scrapes beautiful soup tables, appends to list of tables.
    
    soup: beautiful soup find_all() object
    
    """ 
    
    filing_dfs = []
    for t in soup_tables:
   
        if len(t) > 3:
            table_rows = t.find_all('tr')

            res = []
            for tr in table_rows:
                td = tr.find_all('td')
                row = [tr.text.strip() for tr in td if tr.text.strip()]
                if row:
                    res.append(row)
            df = pd.DataFrame(res)
            filing_dfs.append(df)
        
    print("There are",len(filing_dfs), 'tables in the filing')
    return filing_dfs

# helper function to find table query terms
def table_term_finder(x, search_terms):
    for table_term in x:
        if table_term:
            for sterm in search_terms:
                if str(sterm).lower() in str(table_term).lower():
                    return 'yes'
                
def get_search_tables(search_terms, filing_dfs):
    """
    Arg: search terms list
    
    Returns: 
    List of tables (dataframes) that meet search criteria
    
    """
    interested = []
    for table in filing_dfs:
        # return True if term found
        presence = table.apply(lambda x: table_term_finder(x, search_terms), axis = 1) 

        # append relvant tables from sec filing
        for response in presence:
            if response:
                interested.append(table)
    return interested



def clean_report_table(single_df, new_cols):
    """
    Takes in a single dataframe and cleans, concatenates for export ready df
    
    Params:
    
    single_df: a df (table) from an html sec filing
    new_cols: list of new col names
    
    Returns:
    
    pandas dataframe
    
    """
    
    
    df = single_df.copy()
    
    # column 1 is populated with shifted, isolated dollar symbol, fixed here
    usd_df = df[df[1] == '$']
    no_usd_df = df[df[1] != '$']

    to_drop = []
    for col in usd_df:
        for var in usd_df[col]:
            if '$' in var:
                to_drop.append(col)
                break

    # Drop columns
    usd_df.drop(columns = to_drop, inplace =True)

    # realign columns after drops
    new_cols = [num for num in range(len(usd_df.columns))]
    usd_df.rename(columns = {col:new_col for col, new_col in zip(usd_df.columns, new_cols)},inplace = True)

    # drop all columns with None entirely
    no_usd_df.dropna(axis =1, how ='all', inplace = True)

    # concat new df
    newdf = pd.concat([no_usd_df, usd_df])
    newdf.sort_index(inplace = True)
    
    # define new columns
    newdf.rename(columns = {old:newcol for old,newcol in zip(newdf.columns, newcols)}, inplace = True)
    
    # clean strings, convert to float
    newdf.set_index(0, inplace = True)
    

    return newdf


# Get CIK Numbers and Filings Data

In [3]:
# List of company names to search
co_list = ['Microsoft Corp', 'Amazon Com']

# result dictionaries from CIK retrieval and filings data retrieval functions
co_dict = get_ciks(co_list)
co_dict = get_filings_data(co_dict)

In [4]:
filings_df = pd.DataFrame.from_dict(co_dict, orient ='index')
filings_df.head()

,cik,cik_link,f_type,f_date,f_num,acc_num
Microsoft Corp,0000789019,https://www.sec.gov/cgi-bin/browse-edgar?actio...,"[10-K, 10-K, 10-K, 10-K, 10-K, 10-K, 10-K, 10-...","[2019-08-01, 2018-08-03, 2017-08-02, 2016-07-2...","[001-3784519992755, 001-3784518990758, 001-378...","[0001564590-19-027952, 0001564590-18-019062, 0..."
Amazon Com,0001018724,https://www.sec.gov/cgi-bin/browse-edgar?actio...,"[10-K, 10-K, 10-K, 10-K, 10-K, 10-K, 10-K, 10-...","[2020-01-31, 2019-02-01, 2018-02-02, 2017-02-1...","[000-2251320562951, 000-2251319557427, 000-225...","[0001018724-20-000004, 0001018724-19-000004, 0..."


# Navigate the SEC filing landing page for co

Build dataframe table with associated data per requested file

In [5]:
# company of interest
co = 'Microsoft Corp'
cik = filings_df.loc[co,'cik']
# takes company name and number of filing -- 0 is the most recent filing
co_filing_link = filing_homelink(co, 0)
# index 0 is primary filing data table
files_table = filing_landing_tables(co_filing_link)[0]

files_table

,0,1,2,3,4
0,1,10-K,msft-10k_20190630.htm,10-K,5960450
1,2,EX-4.16,msft-ex416_464.htm,EX-4.16,99733
2,3,EX-10.13,msft-ex1013_465.htm,EX-10.13,213767
3,4,EX-21,msft-ex21_9.htm,EX-21,10972
4,5,EX-23.1,msft-ex231_10.htm,EX-23.1,2465
5,6,EX-31.1,msft-ex311_13.htm,EX-31.1,8084
6,7,EX-31.2,msft-ex312_12.htm,EX-31.2,8242
7,8,EX-32.1,msft-ex321_7.htm,EX-32.1,5077
8,9,EX-32.2,msft-ex322_6.htm,EX-32.2,5339
9,Complete submission text file,0001564590-19-027952.txt,26807783,None,None


# Build dfs from tables in SEC filing

In [6]:
# get file path for html filing
html_file = get_html_path(files_table, '10-K', co, 0)
html_text = requests.get(html_file).text

# Build beautiful soup object
soup = bs4(html_text, 'lxml')
# retrieve all tables from BeautifulSoup object
soup_tables = soup.find_all('table')

# filing tables
filing_dfs = get_filing_tables(soup_tables)

There are 89 tables in the filing


# Find and store tables of interest in filing

* Populate search_terms list with relevant requirements for data tables
* e.g. 'operating income', 'revenue', 'current assets', etc...

In [7]:
# Define search terms for filing tables
search_terms = ['Weighted average shares outstanding']
searched_tables = get_search_tables(search_terms, filing_dfs)

# inspect tables -- identify wanted table
# searched_tables

# Clean data frames 

* USD dollar symbol creates awkward column staggering
* Handled by splitting two dataframes and concatenating after clean-up

In [13]:
# define new columns -- typically reporting periods
newcols = [0, 2019, 2018, 2017]
inc_state = searched_tables[0]
df = clean_report_table(inc_state, newcols)
df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:475: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

,2019,2018,2017
0,,,
"(In millions, except per share amounts)",None,None,None
"Year Ended June 30,",2019,2018,2017
Revenue:,None,None,None
Product,"66,069","64,497","63,811"
Service and other,"59,774","45,863","32,760"


# Additional df clean-up - as needed

In [14]:
# define special characters to replace / withold 
chars_replace = [',', '(', ')', "''"]
for char in chars_replace:
    df = df.apply(lambda x: x.str.replace(char, ''), axis =1)

df.fillna(0, inplace = True)
df = df.astype('float')
df.drop(index = df.index[:2], inplace = True)

df

,2019,2018,2017
0,,,
Revenue:,0.00,0.00,0.00
Product,66069.00,64497.00,63811.00
Service and other,59774.00,45863.00,32760.00
Total revenue,125843.00,110360.00,96571.00
Cost of revenue:,0.00,0.00,0.00
Product,16273.00,15420.00,15175.00
Service and other,26637.00,22933.00,19086.00
Total cost of revenue,42910.00,38353.00,34261.00
Gross margin,82933.00,72007.00,62310.00


# Example run with Microsoft 10-K

* Compare to the 10-K image

<img src="msft_snap.jpg">

In [295]:
# define indices
ind = df.index

# IS items
yoy_revtotal = df.loc['Total revenue']
yoy_gpmarg = df.loc['Gross margin']
yoy_ebit = df.loc['Operating income']
yoy_ni = df.loc['Net income']

# Revenues

In [296]:
yoy_revs = df.iloc[ind.get_loc('Revenue:'): ind.get_loc('Total revenue')]
yoy_revs

,2019,2018,2017
0,,,
Revenue:,0.0,0.0,0.0
Product,66069.0,64497.0,63811.0
Service and other,59774.0,45863.0,32760.0


# Cost of Revenue

In [297]:
yoy_cogs = df.iloc[ind.get_loc('Cost of revenue:'): ind.get_loc('Total cost of revenue')]
yoy_cogs

,2019,2018,2017
0,,,
Cost of revenue:,0.0,0.0,0.0
Product,16273.0,15420.0,15175.0
Service and other,26637.0,22933.0,19086.0


# Earning per Share

In [298]:
yoy_eps = df.iloc[ind.get_loc('Earnings per share:'): ind.get_loc('Weighted average shares outstanding:')]
yoy_eps

,2019,2018,2017
0,,,
Earnings per share:,0.00,0.00,0.00
Basic,5.11,2.15,3.29
Diluted,5.06,2.13,3.25


# Ratios

In [299]:
# Gross profit margins
gpmargins = yoy_gpmarg / yoy_revtotal
gpmargins

2019    0.659020
2018    0.652474
2017    0.645225
dtype: float64

# Operating Expense as % Revenue

In [300]:
yoy_opex = df.iloc[ind.get_loc('Gross margin')+1: ind.get_loc('Operating income')]
opexp = yoy_opex / yoy_revtotal
opexp

,2019,2018,2017
0,,,
Research and development,0.134104,0.133436,0.134999
Sales and marketing,0.144728,0.158291,0.160100
General and administrative,0.038818,0.043077,0.046401
Restructuring,0.000000,0.000000,0.003169


# Export to spreadsheet

In [15]:
df.to_csv('msft_is_10k2019.csv')